In [7]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import warnings

In [5]:
# Download data AAPL (Apple)

data = yf.download("AAPL", start = "2023-01-01", end = '2024-01-01', progress=False)

data.to_csv("APPL_prices.csv")

data.head()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2023-01-03,123.632530,129.395518,122.742873,128.782649,112117500
2023-01-04,124.907707,127.181276,123.642420,125.431615,89113600
2023-01-05,123.583107,126.301500,123.326101,125.668857,80962700
2023-01-06,128.130203,128.792501,123.454572,124.561702,87754700
2023-01-09,128.654144,131.876686,128.397138,128.970474,70790800


In [14]:
# Cleaning and scaling

from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

scaler = MinMaxScaler(feature_range=(0,1))

data.fillna(method='ffill', inplace=True)

data = data[['Close']]

data['Scaled Close'] = scaler.fit_transform(data['Close'])

data.head()


Price,Close,Scaled Close
Ticker,AAPL,
Date,,
2023-01-03,123.632530,0.000674
2023-01-04,124.907707,0.018060
2023-01-05,123.583107,0.000000
2023-01-06,128.130203,0.061996
2023-01-09,128.654144,0.069140


In [26]:
# Convert data into sequences

sequence_length = 10

def create_sequences(data, seq_length):
    sequences = []
    targets = []

    for i in range(len(data)- seq_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(data[i + sequence_length])

    return np.array(sequences), np.array(targets)

X_APPL, y_APPL = create_sequences(data['Scaled Close'].values, sequence_length)

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split